Notebook to perform sensitivity analysis on either direct simulations or emulations

In [1]:
# Import relevant libraries
import os
import pandas as pd

from SALib.analyze.sobol import analyze
import matplotlib.pyplot as plt
import pickle
import pandas as pd

from comparative_gsa.utils import helpers

import torch

from autoemulate.core.sensitivity_analysis import SensitivityAnalysis
from autoemulate.core.sensitivity_analysis import _sobol_results_to_df 

figsize = (9, 5)

In [2]:
param_path = '../inputs/parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas.json'
# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

n_saltelli_samples = 61440

n_samples_used_for_emulator = 256

simulation_out_path = f'../outputs/simulations/output_{n_saltelli_samples}_samples_{param_filename}/'
emulator_path = f'../outputs/simulations/output_{n_samples_used_for_emulator}_samples_{param_filename}/emulators/'

In [3]:
# Load the problem pickle file
problem_path = os.path.join(simulation_out_path, 'problem.pkl')

with open(problem_path, 'rb') as f:
    problem = pickle.load(f)

problem



{'num_vars': 14,
 'names': ['ao.r',
  'ao.c',
  'art.r',
  'art.c',
  'ven.r',
  'ven.c',
  'av.r',
  'mv.r',
  'la.E_pas',
  'la.E_act',
  'la.k_pas',
  'lv.E_pas',
  'lv.E_act',
  'lv.k_pas'],
 'bounds': array([[1.95000000e+02, 3.24999996e+02],
        [2.25000007e-01, 3.74999993e-01],
        [8.43750051e+02, 1.40625000e+03],
        [2.25000011e+00, 3.74999994e+00],
        [6.75000004e+00, 1.12499995e+01],
        [8.49750003e+01, 1.41624995e+02],
        [4.50000008e+00, 7.49999965e+00],
        [3.07500007e+00, 5.12499994e+00],
        [2.25000008e-01, 3.74999990e-01],
        [4.12500030e-01, 6.87499985e-01],
        [6.66000059e-03, 2.99999983e-02],
        [7.50000038e-01, 1.25000000e+00],
        [2.25000008e+00, 3.74999997e+00],
        [6.66000117e-03, 2.99999989e-02]]),
 'sample_scaled': True}

In [4]:
data_type = 'emulations' # 'simulations' or 'emulations'

# Load the simulation summary results
simulations_summary = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

# Load the parameter values
parameters_csv = pd.read_csv(os.path.join(simulation_out_path,'saltelli_samples.csv'))

# Turn x into a pytorch tensor, for emulators to predict on
x = torch.tensor(parameters_csv.values, dtype=torch.float32)

# Loop over each feature
for i_output_feature in simulations_summary.columns:

    print(f'Analysing {i_output_feature} with {data_type}')

    if data_type == 'simulations':

        Y_feature = simulations_summary[i_output_feature].values

    elif data_type == 'emulations':

        # Predict the Y_feature using the emulator

        # Load the right emulator
        emulator = helpers.ae_load_result(os.path.join(emulator_path, 
                                                       i_output_feature, 
                                                       'GaussianProcess_0'))

        # Use the input file to predict with the emulator
        Y_feature = emulator.model.predict(x).mean

        # Turning the Y_feature into numpy
        Y_feature = Y_feature.detach().cpu().numpy().squeeze()

    # Do the sobol_analyse for GSA
    sobol_indices = analyze(problem, Y_feature, calc_second_order=True)
    
    # Transform to a dataframe
    results = {
        i_output_feature: sobol_indices
    }    
    sobol_df = _sobol_results_to_df(results)

    # Swap S1 ST due to bug
    print('Warning, swapping S1 and ST because of a bug in Autoemulate plotting')
    sobol_df = helpers.swap_s1_st(sobol_df=sobol_df)

    # Save the sobol_indices and sobol df
    save_dir = os.path.join(f'../outputs/sa_results/saltelli_samples_{n_saltelli_samples}_{param_filename}/{i_output_feature}/{data_type}/trained_on_{n_samples_used_for_emulator}_samples')
    os.makedirs(save_dir, exist_ok=True)
    sobol_df.to_csv(os.path.join(save_dir,'sobol_df.csv'), index=False)

    # Plot the figures and save
    fig_s1 = SensitivityAnalysis.plot_sobol(sobol_df, index="S1", figsize=figsize) 
    fig_s1.savefig(os.path.join(save_dir, f's1_indices.png'), 
               dpi=300, bbox_inches='tight')
    plt.close(fig_s1)  # Close the figure to free memory
    fig_st = SensitivityAnalysis.plot_sobol(sobol_df, index="ST", figsize=figsize) 
    fig_st.savefig(os.path.join(save_dir, f'st_indices.png'), 
               dpi=300, bbox_inches='tight')    
    plt.close(fig_st)  # Close the figure to free memory

Analysing v_ao_mean with emulations


/Users/lbokeria/projects/health_gc/cvd_net_project/comparative-gsa/venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


14
Warning, swapping S1 and ST because of a bug in Autoemulate plotting
Analysing v_ao_max with emulations


/Users/lbokeria/projects/health_gc/cvd_net_project/comparative-gsa/venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


14
Warning, swapping S1 and ST because of a bug in Autoemulate plotting


Failed to load model from ../outputs/simulations/output_256_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/emulators/v_ao_min/GaussianProcess_0.joblib: [Errno 2] No such file or directory: '../outputs/simulations/output_256_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/emulators/v_ao_min/GaussianProcess_0.joblib'


Analysing v_ao_min with emulations


FileNotFoundError: [Errno 2] No such file or directory: '../outputs/simulations/output_256_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/emulators/v_ao_min/GaussianProcess_0.joblib'